In [1]:
!pip -q install -U trl transformers datasets accelerate peft torch huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import login
login()  # paste your HF token


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

SFT_MODEL_ID = "BekiTila/bitext-sft-tinyllama-support"

tokenizer = AutoTokenizer.from_pretrained(SFT_MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

sft_model = AutoModelForCausalLM.from_pretrained(
    SFT_MODEL_ID,
    device_map="auto",
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)
sft_model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/410 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=

In [5]:
import json
from tqdm import tqdm
from datasets import load_dataset

DATASET_NAME = "bitext/Bitext-customer-support-llm-chatbot-training-dataset"
raw = load_dataset(DATASET_NAME)["train"].shuffle(seed=42)

print("Columns:", raw.column_names)
print("Example:", raw[0])

def get_prompt(ex):
    for k in ["instruction","prompt","query","question","customer_message","text","input"]:
        if k in ex and isinstance(ex[k], str) and ex[k].strip():
            return ex[k]
    for k, v in ex.items():
        if isinstance(v, str) and v.strip():
            return v
    return None

def infer_prompt(p):
    return f"### Instruction:\n{p}\n\n### Response:\n"

@torch.inference_mode()
def sample_answer(prompt_text, temperature):
    inputs = tokenizer(infer_prompt(prompt_text), return_tensors="pt").to(sft_model.device)
    out = sft_model.generate(
        **inputs,
        max_new_tokens=220,
        do_sample=True,
        temperature=temperature,
        top_p=0.95,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    full = tokenizer.decode(out[0], skip_special_tokens=True)
    return full.split("### Response:\n", 1)[-1].strip()

def score(ans: str) -> int:
    # Simple “support answer quality” heuristic (good enough to learn DPO)
    s = 0
    s += min(len(ans), 2000)
    if "\n" in ans: s += 50
    if "1." in ans or "1)" in ans: s += 80
    if "step" in ans.lower(): s += 40
    if "please" in ans.lower() or "sorry" in ans.lower(): s += 15
    return s

N = 200
pairs = []

for ex in tqdm(raw.select(range(N))):
    p = get_prompt(ex)
    if not p:
        continue
    a1 = sample_answer(p, temperature=0.7)  # usually better
    a2 = sample_answer(p, temperature=1.2)  # usually worse

    if not a1 or not a2 or a1 == a2:
        continue

    if score(a1) >= score(a2):
        chosen, rejected = a1, a2
    else:
        chosen, rejected = a2, a1

    pairs.append({"prompt": p, "chosen": chosen, "rejected": rejected})

print("Preference pairs:", len(pairs))

with open("prefs_train.jsonl", "w", encoding="utf-8") as f:
    for r in pairs:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print("Saved prefs_train.jsonl")


Columns: ['flags', 'instruction', 'category', 'intent', 'response']
Example: {'flags': 'BILQ', 'instruction': 'what do i need to do to change to the gold account', 'category': 'ACCOUNT', 'intent': 'switch_account', 'response': "To make the transition to our esteemed {{Account Type}} account, let me guide you through the steps:\n\n1. Access Your Account: Begin by logging in to our platform.\n2. Account Settings: Once inside, locate the '{{Settings}}' or '{{Profile}}' section.\n3. Upgrade Option: Look for an option labeled '{{Upgrade Account}}' or '{{Account Change}}' within these sections.\n4. Select {{Account Type}}: Choose the '{{Account Type}}' option from the available choices.\n5. Finalize the Switch: Follow the on-screen prompts to complete the transition.\n\nIf you encounter any hurdles or have further inquiries, feel free to reach out. We value your decision to upgrade, and I'm here to ensure a smooth and rewarding experience. Enjoy the exclusive benefits that come with the {{Ac

100%|██████████| 200/200 [19:55<00:00,  5.98s/it]

Preference pairs: 200
Saved prefs_train.jsonl


In [6]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig

dpo_ds = load_dataset("json", data_files="prefs_train.jsonl")["train"]

# Policy model starts from SFT
policy_model = AutoModelForCausalLM.from_pretrained(
    SFT_MODEL_ID,
    device_map="auto",
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

# Reference model is a frozen copy (same weights as SFT at start)
ref_model = AutoModelForCausalLM.from_pretrained(
    SFT_MODEL_ID,
    device_map="auto",
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)

OUT_DIR = "outputs/dpo"

cfg = DPOConfig(
    output_dir=OUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    learning_rate=5e-6,
    logging_steps=25,
    save_steps=200,
    bf16=torch.cuda.is_available(),
    report_to="none",
    max_length=512,
    max_prompt_length=256,
)

trainer = DPOTrainer(
    model=policy_model,
    ref_model=ref_model,
    args=cfg,
    train_dataset=dpo_ds,
    processing_class=tokenizer,   # IMPORTANT for TRL 0.26.x
)

trainer.train()
trainer.save_model(OUT_DIR)
tokenizer.save_pretrained(OUT_DIR)

print("Saved DPO model to:", OUT_DIR)


Generating train split: 0 examples [00:00, ? examples/s]

Extracting prompt in train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss


Saved DPO model to: outputs/dpo


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

DPO_REPO_ID = "BekiTila/bitext-dpo-tinyllama-support"  # new repo name

tok = AutoTokenizer.from_pretrained("outputs/dpo")
mdl = AutoModelForCausalLM.from_pretrained("outputs/dpo")

tok.push_to_hub(DPO_REPO_ID)
mdl.push_to_hub(DPO_REPO_ID)

print("Pushed DPO model to:", DPO_REPO_ID)


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...p9b_mvfgl/tokenizer.model: 100%|##########|  500kB /  500kB            

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...9xx6zn1/model.safetensors:   0%|          | 1.19MB / 4.40GB            

Pushed DPO model to: BekiTila/bitext-dpo-tinyllama-support


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

@torch.inference_mode()
def run(model_id, prompt):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    mdl = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    )
    inp = tok(f"### Instruction:\n{prompt}\n\n### Response:\n", return_tensors="pt").to(mdl.device)
    out = mdl.generate(**inp, max_new_tokens=200, do_sample=True, temperature=0.7, top_p=0.9)
    full = tok.decode(out[0], skip_special_tokens=True)
    return full.split("### Response:\n", 1)[-1].strip()

test_prompt = "My payment failed. What should I check?"
print("SFT:\n", run("BekiTila/bitext-sft-tinyllama-support", test_prompt))
print("\nDPO:\n", run("BekiTila/bitext-dpo-tinyllama-support", test_prompt))


SFT:
 I've picked up that you've encountered a payment failure, and I'm here to assist you in identifying the root cause and finding a solution. To better understand the issue, could you please provide more details about the specific payment method you used? This information will help me investigate and provide you with the most accurate guidance. Your patience and cooperation are greatly appreciated as we work together to resolve this issue.


tokenizer_config.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/410 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]


DPO:
 I'm sorry to hear that your payment failed. To assist you with this issue, please provide me with more details about the specific error message or any error codes you received. This information will help us investigate and resolve the problem more effectively. Your cooperation is greatly appreciated, and we are committed to finding a solution for you.
